# Concept

The goal here is to build a sequential web scraper based off of the TTBID. I think that the simplest thing to do will just be to start with a basic easy date and then just increment until we get an error back. Since things are _supposedly_ sequential, we can interate easily. A good test will be to try for a small test range. It may also be worth trying to get those parallel scraping tools operational

<div class="alert alert-block alert-info">
TTB ID - This is a unique, 14 digit number assigned by TTB to track each COLA.  The first 5 digits represent the calendar year and Julian date the application was received by TTB. The next 3 digits tell how the application was received (001 = e-filed; 002 & 003 = mailed/overnight; 000 = hand delivered). The last 6 digits is a sequential number that resets for each day and for each received code.
</div>

# Imports

In [48]:
import requests
from bs4 import BeautifulSoup
import re

import datetime
import pymongo

### Early prototypes

In [35]:
start_date = '01/01/2016'
stop_date = '01/30/2016'

In [40]:
date_start = datetime.datetime.strptime(start_date, '%m/%d/%Y')
date_stop = datetime.datetime.strptime(stop_date, '%m/%d/%Y')

curr_date = date_start
curr_reccode = 1
curr_seqnum = 1

# prep the strings for the ttbid
jdate='{year}{day}'.format(year=curr_date.strftime('%y'), day=curr_date.strftime('%j'))
reccode='{:03d}'.format(curr_reccode)
seqnum='{:06d}'.format(curr_seqnum)

# prep the query
ttbid = '{jdate}{reccode}{seqnum}'.format(jdate=jdate, reccode=reccode, seqnum=seqnum)
url = r'https://www.ttbonline.gov/colasonline/viewColaDetails.do'
params = {'action': 'publicDisplaySearchBasic',
          'ttbid': ttbid}

# send the query
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text, 'html5lib')

# see if we have the table of data w expect
#trs = soup.select('form[name=colaApplicationForm] div.box tr')  # extract each table row

# if we got a valid response
if trs:
    entry = {'_id': ttbid,
             'rec_code': reccode,
             'seq_num': seqnum}

In [42]:
trs = soup.select('form[name=colaApplicationForm] div.box tr')  # extract each table row

In [46]:
trs

<function list.count>

In [ ]:
def build_ttb_database(start_date, stop_date):

# Connecting to the Mongo DB

In [2]:
# Set up connection to mongodb
client = pymongo.MongoClient() # Connect to default client
db = client.TTB # Get a database (note: lazy evaluation)
TTB = db.TTB # the actual collection

In [4]:
TTB.count() # number of elements in the database

0

In [5]:
a = TTB.distinct('TTBID') # list of distinct TTBID's
len(a)

0